# Proyecto Fin de Máster - Máster en Big Data y Business Analytics V Edición
# Big Data International Campus - Universidad Europea Miguel de Cervantes

### *Eliasib Jesús García Martín*
---

# 1. Introducción

El propósito de este proyecto es el de crear un producto de datos que sigan las metodologías de análisis de datos que se han impartido a lo largo de este máster. 

Para ello, se ha propuesto como conjunto de datos (en adelante, dataset) sobre los que aplicar dicho análisis el conjunto sobre datos de uso de un sistema de alquiler de bicicletas en el área de la Bahía de San Francisco (SF Bay Area Bike Share - https://www.kaggle.com/benhamner/sf-bay-area-bike-share).

En cuanto al análisis a realizar, se centrará en desarrollar, utilizando las metodologías más adecuadas, los siguientes puntos clave:
* Investigar la influencia del tiempo atmosférico en los viajes en bicicletas registrados en el dataset.
* Reconocer gracias un modelo de clasificación patrones en las rutas que realizan los usuarios, en función de las estaciones de origen y de destino en viajes realizados en horarios similares.

# 2. Carga de los datos y análisis descriptivo

El dataset sobre el que se basa este proyecto está compuesto por cuatro registros de datos, en forma de ficheros CSV (Comma Separated Values). La información contenida en estos cuatro ficheros está interrelacionada, por lo que la unión de todos ellos conforma el conjunto completo de los datos.

Los ficheros de los que se compone el dataset son:
* **station.csv:** En este fichero se listan las estaciones donde los usuarios pueden alquilar y/o depositar las bicicletas. Las columas para cada registro son:
    * **id** - Identificador del registro
    * **name** -  Nombre de la estación
    * **lat** - Latitud de la ubicación de la estación
    * **lon** - Longitud de la ubicación de la estación
    * **dock_count** - Número de plazas de bicicletas que puede alojar la estación
    * **city** - Ciudad donde se encuentra la estación
    * **installation_date** - Fecha de instalación de la estación


* **weather.csv:** En este fichero se listan los datos del tiempo atmosférico registrados diariamente. Las columnas para cada registro son:
    * **date** - Fecha del registro
    * **max_temperature_f** - Temperatura máxima en grados Fahrenheit
    * **mean_temperature_f** - Temperatura media en grados Fahrenheit
    * **min_temperature_f** - Temperatura mínima en grados Fahrenheit
    * **max_dew_point_f** - Temperatura máxima de rocío en grados Fahrenheit
    * **mean_dew_point_f** - Temperatura media de rocío en grados Fahrenheit
    * **min_dew_point_f** - Temperatura mínima de rocío en grados Fahrenheit
    * **max_humidity** - Humedad máxima
    * **mean_humidity** - Humedad media
    * **min_humidity** - Humedad mínima
    * **max_sea_level_pressure_inches** - Presión atmosférica máxima a nivel del mar en pulgadas de mercurio
    * **mean_sea_level_pressure_inches** - Presión atmosférica media a nivel del mar en pulgadas de mercurio
    * **min_sea_level_pressure_inches** - Presión atmosférica mínima a nivel del mar en pulgadas de mercurio
    * **max_visibility_miles** - Visibilidad máxima en millas
    * **mean_visibility_miles** - Visibilidad media en millas
    * **min_visibility_miles** - Visibilidad mínima en millas
    * **max_wind_speed_mph** - Velocidad máxima del viento en millas por hora
    * **mean_wind_speed_mph** - Velocidad media del viento en millas por hora
    * **min_wind_speed_mph** - Velocidad mínima del viento en millas por hora
    * **precipitation_inches** - Precipitaciones en pulgadas
    * **cloud_cover** - Nubosidad en octas
    * **events** - Eventos atmosféricos (lluvia, niebla, tormenta...)
    * **wind_dir_degrees** - Dirección del viento en grados
    * **zip_code** - Código postal de la localización del registro


* **trip.csv:** En este fichero se listan los viajes en bicicleta registrados. Las columnas para cada registro son:
    * **id** - Identificador del registro
    * **duration** - Duración del viaje en segundos
    * **start_date** - Fecha y hora de inicio del viaje
    * **start_station_name** - Nombre de la estación donde se inició el viaje
    * **start_station_id** - Identificador de la estación donde se inició el viaje
    * **end_date** - Fecha y hora de finalización del viaje
    * **end_station_name** - Nombre de la estación donde se finalizó el viaje
    * **end_station_id** - Identificador de la estación donde se finalizó el viaje
    * **bike_id** - Identificador de la bicicleta utilizada para el viaje


* **status.csv:** En este fichero se listan registros por minuto del estado de las estaciones. Las columnas para cada registro son:
    * **station_id** - Identificador de la estación a la que se refiere el registro
    * **bikes_available** - Número de bicicletas disponibles
    * **docks_available** - Número de plazas de estacionamiento para bicicletas disponibles
    * **time** - Fecha y hora del registro
    


Puesto que el dataset se encuentra alojado en Kaggle, debemos descargar los ficheros CSV que lo componen, pero para ello se requiere la creación de una cuenta gratuita en Kaggle. Una vez se ha accedido a la plataforma, se puede descargar el dataset comprimido en formato zip a través de este enlace: https://www.kaggle.com/benhamner/sf-bay-area-bike-share/downloads/sf-bay-area-bike-share.zip

Una vez obtenido el fichero zip, para poder trabajar con los datos desde este notebook, se debe descomprimir los ficheros CSV bajo una carpeta `data/` relativa a la ruta donde se encuentre este notebook.

También debe tenerse en cuenta que para la carga de datos se utiliza el paquete Pandas, por lo que debe estar instalado para que el siguiente código se ejecute correctamente.

In [2]:
import pandas as pd

# Se cargan los ficheros CSV como DataFrames de Pandas
station_df = pd.read_csv('data/station.csv')
weather_df = pd.read_csv('data/weather.csv')
trip_df = pd.read_csv('data/trip.csv')
status_df = pd.read_csv('data/status.csv')

La librería Pandas también facilita bastante el análisis descriptivo de los datos, con funciones que calculan y muestran de manera ordenada valiosa información sobre los DataFrames. Un ejemplo es la función `.describe()`, gracias a la cual pueden verse tabulados valores como el máximo, el mínimo, la media o la desviación típica, entre otros, de las variables numéricas de los DataFrames.

In [6]:
# Describing station DataFrame
station_df.describe()

,id,lat,long,dock_count
count,70.000000,70.000000,70.000000,70.000000
mean,43.000000,37.590243,-122.218416,17.657143
std,24.166092,0.203473,0.209446,4.010442
min,2.000000,37.329732,-122.418954,11.000000
25%,24.250000,37.389483,-122.400601,15.000000
50%,43.500000,37.631163,-122.312123,15.000000
75%,63.750000,37.788123,-122.078009,19.000000
max,84.000000,37.804770,-121.877349,27.000000


Como información relevante en el caso de las estaciones, se observa que con las latitudes y longitudes máximas y mínimas podríamos por ejemplo encuadrar en un mapa el área donde se situan la totalidad de las estaciones registradas. También puede ser útil conocer el número medio de plazas para bicicletas que hay en las estaciones, aunque habría que revisar si la desviación estándar es muy elevada, en cuyo caso la media no sería un valor muy significativo.

In [7]:
# Describing weather DataFrame
weather_df.describe()

,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure_inches,...,min_sea_level_pressure_inches,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,cloud_cover,wind_dir_degrees,zip_code
count,3661.000000,3661.000000,3661.000000,3611.000000,3611.000000,3611.000000,3611.000000,3611.000000,3611.000000,3664.000000,...,3664.000000,3652.000000,3652.000000,3652.000000,3664.000000,3664.000000,2766.000000,3664.000000,3664.000000,3665.000000
mean,70.580989,61.348812,51.947282,53.149820,48.794794,44.306840,85.446968,67.256993,46.458045,30.074997,...,29.965835,10.555312,9.822015,8.229737,16.398472,5.962063,22.299349,2.784389,266.605895,94325.000000
std,8.385572,7.234397,7.441444,6.885449,7.822281,9.355432,9.226972,10.945591,14.434784,0.134492,...,0.133476,2.193828,1.469894,2.862132,7.787588,3.340603,9.618813,2.276401,102.047645,404.615212
min,44.000000,38.000000,25.000000,20.000000,13.000000,2.000000,24.000000,24.000000,4.000000,29.500000,...,28.980000,5.000000,4.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,94041.000000
25%,64.000000,56.000000,47.000000,49.000000,44.000000,39.000000,82.000000,62.000000,37.000000,29.980000,...,29.880000,10.000000,10.000000,7.000000,13.000000,3.000000,17.000000,1.000000,254.000000,94063.000000
50%,70.000000,61.000000,53.000000,54.000000,49.000000,46.000000,86.000000,68.000000,48.000000,30.060000,...,29.950000,10.000000,10.000000,10.000000,16.000000,6.000000,22.000000,3.000000,297.000000,94107.000000
75%,77.000000,67.000000,58.000000,58.000000,55.000000,52.000000,93.000000,74.000000,56.000000,30.170000,...,30.050000,10.000000,10.000000,10.000000,20.000000,8.000000,25.000000,5.000000,330.000000,94301.000000
max,102.000000,84.000000,75.000000,68.000000,65.000000,63.000000,100.000000,96.000000,93.000000,30.650000,...,30.370000,20.000000,20.000000,20.000000,128.000000,23.000000,114.000000,8.000000,2772.000000,95113.000000


En el caso de las temperaturas registradas, la propia estructura de los datos ya separaba la mayoría de su información por valores máximos, medios y mínimos, pero la aplicación de la función `.describe()` sobre este DataFrame también arroja nuevo datos interesantes, cómo cuáles fueron los valores de temperatura, humedad, presión atmosférica, velocidad del viento etc. máximos y mínimos alcanzados durante el tiempo que se registraron datos. También podemos comprobar gracias al cálculo de la desviación estándar y a su pequeño valor que el componente con menor variación en los registros fue el de la presión atmosférica.

In [8]:
# Describing trip DataFrame
trip_df.describe()

,id,duration,start_station_id,end_station_id,bike_id
count,669959.000000,6.699590e+05,669959.000000,669959.000000,669959.000000
mean,460382.009899,1.107950e+03,57.851876,57.837438,427.587620
std,264584.458487,2.225544e+04,17.112474,17.200142,153.450988
min,4069.000000,6.000000e+01,2.000000,2.000000,9.000000
25%,231082.500000,3.440000e+02,50.000000,50.000000,334.000000
50%,459274.000000,5.170000e+02,62.000000,62.000000,440.000000
75%,692601.000000,7.550000e+02,70.000000,70.000000,546.000000
max,913460.000000,1.727040e+07,84.000000,84.000000,878.000000


En el caso de los viajes registrados, la única variable no categórica, y por tanto de la que podemos extraer información con la función `.describe()`, es la duración de los viajes en segundos, y resulta algo incómodo ver valores con exponentes. Por ello, se puede aplicar una transformación sobre los datos para obtener estos valores de duración en minutos y facilitar la observación de los valores de esta variable.

In [10]:
# Adding a new column to trip DataFrame, calculating per each 'duration' value its equivalence in minutes.
trip_df['duration_minutes'] = trip_df['duration'] / 60.0

# Describing trip DataFrame
trip_df.describe()

,id,duration,start_station_id,end_station_id,bike_id,duration_minutes
count,669959.000000,6.699590e+05,669959.000000,669959.000000,669959.000000,669959.000000
mean,460382.009899,1.107950e+03,57.851876,57.837438,427.587620,18.465831
std,264584.458487,2.225544e+04,17.112474,17.200142,153.450988,370.923950
min,4069.000000,6.000000e+01,2.000000,2.000000,9.000000,1.000000
25%,231082.500000,3.440000e+02,50.000000,50.000000,334.000000,5.733333
50%,459274.000000,5.170000e+02,62.000000,62.000000,440.000000,8.616667
75%,692601.000000,7.550000e+02,70.000000,70.000000,546.000000,12.583333
max,913460.000000,1.727040e+07,84.000000,84.000000,878.000000,287840.000000


Tras añadir una nueva columna con la duración de los viajes expresados en minutos, se ha clarificado la información sobre la duración de los viajes registrados, aunque por ejemplo, el valor tan elevado de la desviación estándar indica que la media no es un buen valor sobre el que apoyarse. Otro dato llamativo es que el viaje de mayor duración se ha extendido por 287840 minutos, lo que a la conversión suponen más de seis meses y medio.

In [11]:
# Describing status DataFrame
status_df.describe()

,station_id,bikes_available,docks_available
count,7.198443e+07,7.198443e+07,7.198443e+07
mean,4.253149e+01,8.394812e+00,9.284729e+00
std,2.376117e+01,3.993586e+00,4.175442e+00
min,2.000000e+00,0.000000e+00,0.000000e+00
25%,2.400000e+01,6.000000e+00,6.000000e+00
50%,4.200000e+01,8.000000e+00,9.000000e+00
75%,6.300000e+01,1.100000e+01,1.200000e+01
max,8.400000e+01,2.700000e+01,2.700000e+01


Por último, en cuanto al estado de las estaciones, a nivel descriptivo los datos recogidos no aportan mucha información relevante, pero ha de comprenderse que el valor de los datos recogidos en este DataFrame radica en su variación en el tiempo, y no en los valores de los datos por sí solos.